In [1]:
import datetime as dt

import panel as pn
import pandas as pd

import hvplot.pandas
hvplot.extension('bokeh')


def get_series(**options):
    df = pd.read_csv(options["url"], index_col=0)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(options["tz"])
    df = df.sort_index(ascending=True)
    return df[[options["column"]]].rename(columns={options["column"]: options["name"]})


def get_dataframe(**options):
    df = pd.read_csv(options["url"], index_col=0)
    df.index = pd.to_datetime(df.index, utc=True).tz_convert(options["tz"])
    df = df.sort_index(ascending=True)
    return df


pn.config.theme = 'dark'

height=350
width=600

start_time = dt.datetime.now(dt.UTC) - dt.timedelta(days=7)
start_time

datetime.datetime(2024, 5, 21, 20, 54, 50, 394849, tzinfo=datetime.timezone.utc)

In [2]:
options = dict(
    url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/gridwatch.ca/hourly/summary.csv",
    tz="America/Toronto",
    column="",
)

df = get_dataframe(**options)
df = df[df.index > start_time]

In [3]:
plot_options = dict(
    value_label='MW',
    legend='bottom',
    title="Ontario grid supply and demand",
    height=height,
    width=width,
    grid=True,
)

columns = {x: x.replace(" (MW)", "") for x in df.columns if x.endswith(" (MW)") and (x[0]!=x[0].lower())}
suply_demand = df[columns.keys()].rename(columns=columns).iloc[-24*7:].hvplot.line(**plot_options)
suply_demand

:NdOverlay   [Variable]
   :Curve   [index]   (MW)

In [4]:
plot_options = dict(
    value_label='MW',
    legend='bottom',
    title="Ontario grid generation",
    height=height,
    width=width,
    grid=True,
   
)

columns = {x: x.replace(" (MW)", "") for x in df.columns if x.endswith(" (MW)") and (x[0]==x[0].lower())}
generation = df[columns.keys()].rename(columns=columns).iloc[-24*7:].hvplot.line(**plot_options)
generation

:NdOverlay   [Variable]
   :Curve   [index]   (MW)

In [5]:
plot_options = dict(
    value_label='%',
    legend='bottom',
    title="Ontario grid power mix (%)",
    height=height,
    width=width,
    grid=True,
)

columns = {x: x.replace(" (%)", "") for x in df.columns if x.endswith(" (%)")}
generation_pct = df[columns.keys()].rename(columns=columns).iloc[-24*7:].hvplot.line(**plot_options)
generation_pct

:NdOverlay   [Variable]
   :Curve   [index]   (%)

In [6]:
options_gridwatch = dict(
    name = "gridwatch",
    url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/gridwatch.ca/hourly/summary.csv",
    tz="America/Toronto",
    column = "CO2e Intensity (g/kWh)"
)

options_co2signal = dict(
    name="co2signal",
    url="https://raw.githubusercontent.com/ryanfobel/ontario-grid-data/main/data/clean/co2signal.com/CA-ON/hourly/output.csv",
    column = "data.carbonIntensity",
    tz="America/Toronto",
)

df = get_series(**options_gridwatch).join(
    get_series(**options_co2signal),
    how="inner"
)

plot_options = dict(
    value_label='g/kWh',
    legend='bottom',
    title="Ontario grid co2e intensity",
    height=height,
    width=width,
    grid=True,
)
df = df[df.index > start_time]
co2_intensity = df.hvplot.line(**plot_options)
co2_intensity

:NdOverlay   [Variable]
   :Curve   [index]   (g/kWh)

In [7]:
template = pn.template.BootstrapTemplate(
    title='Ontario grid data',
    main=pn.Column(
        pn.Row(suply_demand, generation),
        pn.Row(generation_pct, co2_intensity),
    )
)
template.servable();

In [8]:
!panel convert index.ipynb --to pyodide-worker --out ..\docs --pwa --title "Ontario energy dashboard"

Successfully converted index.ipynb to pyodide-worker target and wrote output to index.html.
Successfully wrote icons and images.
Successfully wrote site.manifest.
Successfully wrote serviceWorker.js.


http://localhost:8000/docs

In [ ]:
#!python -m http.server